In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Visualising Predictions

### 1. Imports and Model Initialization

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from utils import FinalModel, visualize_prediction
from config_io import save_to_config, get_config_value
from transformers import RobertaModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base",add_prefix_space=True)
text_encoder = RobertaModel.from_pretrained("roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
choices = ["final_weights.pth","best_weights.pth"]      #Choose
i = 1
model_save_weight_path = None
try:
    model_save_weight_path = Path(get_config_value("CHECKPOINTS_PATH")) / choices[i]
except FileNotFoundError as e:
    print(f"Please provide the link for the {choices[i]}")
    model_save_weight_path = Path("/content/drive/MyDrive/refcoco_project/Weights_Checkpoint/best_weights.pth")
    save_to_config({
        "CHECKPOINTS_PATH":str(model_save_weight_path.parent),
    })

In [ ]:
print("Weight file:", model_save_weight_path)

In [ ]:
model = FinalModel(textbackbone=text_encoder)
state_dict = torch.load(model_save_weight_path)
model.load_state_dict(state_dict)
model.eval()

## 3. Evaluate

In [ ]:
phrase = ""
img_path : Path = Path(get_config_value("OUT_DIR",default="not-set")) / ""      #Fill with your folder first and then your

In [ ]:
#Run this cell to see the image before predicted boxes
img = Image.open(img_path).convert("RGB")
img_np = np.array(img)
plt.imshow(img_np)
plt.axis('off')
plt.title("The image")
plt.show()

In [ ]:
visualize_prediction(model,img_path,phrase,tokenizer,device)